In [42]:
import requests
import json
import pandas as pd
import ccxt

In [18]:
## 顯示 json 物件用 Add this to the first block in your note book
import uuid
from IPython.core.display import display, HTML

import json

def RenderJSON(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)


C:\Users\Neoxlarge\AppData\Local\Temp\ipykernel_8868\3362361515.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
symbol_list=["ATOM"]

cg_headers = {'coinglassSecret': '7ef635a6899a4fbdb9db368d222c974d'}
candles = {"1m": 9, "5m": 3, "15m": 10, "30m": 11, "1h": 2, "4h": 1, "12h": 4, "24h": 5, "90d": 18, "all": 0}


In [3]:
# 定義資料取得方法
def get_oi_data(symbol, Timeframe="1m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/openInterest?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)
    
def get_oi_chart_data(symbol, Timeframe):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/openInterest/chart?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_data(symbol, Timeframe="5m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/longShort_chart?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance(symbol, timeframe="5m", limit=24):
    url = f"https://www.binance.com/futures/data/globalLongShortAccountRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance_top_position(symbol, timeframe="5m", limit=24):
    #Top Trader Long/Short Ratio (Position)
    #幣安大戶多空比倉位
    url = f"https://www.binance.com/futures/data/topLongShortPositionRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance_top_account(symbol, timeframe="5m", limit=24):
    #Top Trader Long/Short Ratio (Accounts)
    #幣安大戶多空比帳戶
    url = f"https://www.binance.com/futures/data/topLongShortAccountRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_liq_detial(symbol, Timeframe="5m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/liquidation/detail/chart?symbol={symbol}&timeType={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_fr(symbol, ttype="U"):
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/funding_rates_chart?symbol={symbol}&type={ttype}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def send_to_bot(tid_, msg, bot_token_):
    url = f'https://api.telegram.org/bot{bot_token_}/sendMessage'
    headers = {'content-type': 'application/json'}
    data  = f'{{"chat_id": "{tid_}", "text": "{msg}", "parse_mode": "HTML", "disable_web_page_preview": true}}'.encode('utf-8')
    r = requests.post(url, data=data, headers=headers, verify=False)

In [63]:
exchange = ccxt.binance()
ohlc_data = exchange.fetch_ohlcv(symbol="HNT/USDT",timeframe="1h",limit=91)

cols=["Time","Open","High","Low","Close","Volume"]
ohlc_df = pd.DataFrame(ohlc_data,columns=cols,)

liq_df = pd.DataFrame(columns=["Time","allnet_liq_sell","allnet_liq_buy","Binance_liq_sell","Binance_liq_buy"])
liq_data = get_liq_detial(symbol="HNT",Timeframe="1h")
liq_data = json.dumps(liq_data["data"])
for i in liq_data:
    liq_df.append([0,
                    0,
                    0,0,0])
    



In [53]:
ohlc_df

,Time,Open,High,Low,Close,Volume
0,1663714800000,4.95,5.00,4.90,4.92,149816.19
1,1663718400000,4.92,4.93,4.85,4.87,132685.82
2,1663722000000,4.87,4.88,4.81,4.81,95564.55
3,1663725600000,4.82,4.85,4.77,4.78,71599.79
4,1663729200000,4.78,4.79,4.71,4.74,175463.69
...,...,...,...,...,...,...
86,1664024400000,4.64,4.67,4.60,4.63,108839.76
87,1664028000000,4.63,4.64,4.60,4.61,79900.79
88,1664031600000,4.61,4.62,4.57,4.62,70401.71
89,1664035200000,4.61,4.61,4.57,4.60,36074.88


In [55]:
import os

with open("liq_data.json",mode='w') as f:
    f.write(liq_data)